# **Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

## **Edición 2024**






----

# Encodings

Muchas de las variables que nos encontramos en los conjuntos de datos son categóricas, es decir, variables no numéricas que adquieren valores de un número limitado de clases o categorías. Asimismo, las variables categóricas pueden ser de dos tipos:

***Variables ordinales***: sus valores pueden ser ordenados jerárquicamente, como, por ejemplo, el nivel de educación de una persona (sin estudios, primaria, secundaria o superiores).

***Variables nominales***: no se puede establecer un orden en sus categorías. Algunos ejemplos son el sexo de una persona (hombre o mujer), el color del pelo (castaño, rubio, negro o rojo) o el país de nacimiento (España, Italia, Francia, etc.).
El problema es que la mayoría de los modelos de machine learning en Python, entre ellos las redes neuronales, solo pueden leer valores numéricos puesto que que hacen uso de diversas operaciones matemáticas. Por esto mismo, en estos casos es importante convertir este tipo de variables en numéricas para que los modelos puedan utilizarlas.

Afortunadamente, tenemos a nuestra disposición diversas técnicas fáciles de implementar en Python para codificar este tipo de variables y cada una de estas tiene sus ventajas y desventajas.

**El conjunto de datos**

En los casos prácticos utilizaremos el conjunto de datos público denominado Adult, que pertenece al repositorio UCI Machine Learning. Este conjunto de datos contiene 48.842 instancias que representan datos personales (edad, género, raza, etc.) de ciudadanos anónimos extraídos por Barry Becker de la base de datos del censo de 1994 de los Estados Unidos.

Este dataset es usado para predecir si una persona gana más de 50 mil dólares al año a partir de variables como la edad, género, raza o país de nacimiento de una persona.

Así, para cargar el dataset simplemente empleamos la función read_csv de la librería pandas, a la que le pasamos la ruta donde se alojan los datos y nos devuelve un objeto DataFrame. Una vez tenemos nuestro DataFrame, le asignamos el nombre de cada variable.

In [ ]:
import pandas as pd

# Cargamos el dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning'
                 '-databases/adult/adult.data', encoding = 'utf-8',
                 header = None)

# AÃ±adimos el nombre de cada variable
df.columns = ["age", "workclass", "fnlwgt", "education",
              "education-num", "marital-status", "occupation",
              "relationship", "race", "sex", "capital-gain",
              "capital-loss", "hours-per-week", "native-country",
              "class"]

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Entre las variables independientes encontramos una mezcla de variables numéricas como age o capital-gain y categóricas, tanto de tipo ordinal (education) como nominal (race o workclass).

**Codificación ordinal**

Comenzamos con esta popular técnica de codificación que consiste simplemente en remplazar cada valor de la variable con un número entero distinto. Es por ello que la **codificación ordinal** resulta útil cuando tenemos datos ordinales, es decir, aquellos en los que se puede establecer un orden entre sus categóricas. En consecuencia, esta técnica nos ayudará a representar esta relación de orden de las distintas categorías.

En nuestro conjunto de datos nos centraremos en el atributo ordinal education, en el cual claramente podemos asignar un orden a los distintos niveles de estudios. Además, conocemos ese orden jerárquico que va desde el nivel de estudios más bajo (Preschool) al que el algoritmo le asignará un 0, hasta el nivel máximo de educación (Doctorate) que tendrá asignado un 15.

In [ ]:
df.education.value_counts()


education
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: count, dtype: int64

Como podemos ver en la tabla superior, hay 16 niveles de educación a los que debemos asignar un número entero. Para ello, la librería scikit-learn nos proporciona el objeto OrdinalEncoder para aplicar este tipo de codificación y al cual le podemos indicar el orden de las categorías. Veamos cómo podemos utilizarlo en nuestro dataset.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Creamos el codificador indicandole el orden de la variables
encoder = OrdinalEncoder(categories=[[" Preschool", " 1st-4th",
                                      " 5th-6th", " 7th-8th", " 9th",
                                      " 10th", " 11th", " 12th",
                                      " HS-grad", " Some-college",
                                      " Assoc-voc", " Assoc-acdm",
                                      " Bachelors", " Masters",
                                      " Prof-school", " Doctorate"]])

# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(df[["education"]])
df["education-encoded"] = encoder.transform(df[["education"]])

A continuación, mostramos los valores de la variable antes y después de aplicar la codificación ordinal.

In [ ]:
df[["education", "education-encoded"]].head(10)

,education,education-encoded
0,Bachelors,12.0
1,Bachelors,12.0
2,HS-grad,8.0
3,11th,6.0
4,Bachelors,12.0
5,Masters,13.0
6,9th,4.0
7,HS-grad,8.0
8,Masters,13.0
9,Bachelors,12.0


**Codification one-hot**

Para las variables nominales que no tengan un orden jerárquico en sus categorías, una mejor solución es la llamada **codificación one-hot**.

Esta segunda técnica consiste en crear una nueva **variable binaria** (también llamada dummy) por cada categoría existente en la variable a codificar.  Así, estas nuevas variables contendrán 1 en aquellas observaciones que pertenezcan a esa categoría y 0 en el resto.


In [ ]:
df.race.value_counts()

race
 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Amer-Indian-Eskimo      311
 Other                   271
Name: count, dtype: int64

Vemos que las categorías existentes son: White, Black, Asian-Pac-Islander, Amer-Indian-Eskimo y Other.
Creamos las variables binarias con el método get_dummies y ajustamos el argumento drop_first a True para eliminar primera variable y evitar así los problemas de redundancia.

In [ ]:
# Creamos las variables binarias
dummies = pd.get_dummies(df['race'], drop_first = False)
dummies.head()

,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,White
0,False,False,False,False,True
1,False,False,False,False,True
2,False,False,False,False,True
3,False,False,True,False,False
4,False,False,True,False,False


Ya solo nos queda añadir estas nuevas variables a nuestro DataFrame y eliminar la variable original race.

In [ ]:
# Añadimos las variables binarias al DataFrame
df = pd.concat([df, dummies], axis = 1)

# Eliminamos la vairable original race
df = df.drop(columns=['race'])

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,class,education-encoded,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,White
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K,12.0,False,False,False,False,True
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K,12.0,False,False,False,False,True
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K,8.0,False,False,False,False,True
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K,6.0,False,False,True,False,False
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K,12.0,False,False,True,False,False
